In [1]:
# https://docs.python.org/3.4/installing/index.html
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
import csv
import shutil
import re
import time
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_excel(r'Free+English+textbooks.xlsx')
df.head()

,Book Title,Author,Edition,Product Type,Copyright Year,Copyright Holder,Print ISBN,Electronic ISBN,Language,Language Collection,...,German Package Name,Series Print ISSN,Series Electronic ISSN,Series Title,Volume Number,DOI URL,OpenURL,Subject Classification,Publisher,Imprint
0,Fundamentals of Power Electronics,"Robert W. Erickson, Dragan Maksimovic",2nd ed. 2001,Graduate/advanced undergraduate textbook,2001,Springer Science+Business Media New York,978-0-7923-7270-7,978-0-306-48048-5,EN,English/International,...,NaN,NaN,NaN,NaN,NaN,http://doi.org/10.1007/b100747,http://link.springer.com/openurl?genre=book&is...,"Engineering; Circuits and Systems; Energy, gen...",Springer US,Springer
1,Handbook of the Life Course,"Jeylan T. Mortimer, Michael J. Shanahan",2003,Graduate/advanced undergraduate textbook,2003,Springer Science+Business Media New York,978-0-306-47498-9,978-0-306-48247-2,EN,English/International,...,NaN,1389-6903,2542-839X,Handbooks of Sociology and Social Research,NaN,http://doi.org/10.1007/b100507,http://link.springer.com/openurl?genre=book&is...,"Social Sciences; Sociology, general; Clinical ...",Springer US,Springer
2,All of Statistics,Larry Wasserman,2004,Graduate/advanced undergraduate textbook,2004,Springer-Verlag New York,978-0-387-40272-7,978-0-387-21736-9,EN,English/International,...,NaN,1431-875X,2197-4136,Springer Texts in Statistics,NaN,http://doi.org/10.1007/978-0-387-21736-9,http://link.springer.com/openurl?genre=book&is...,Mathematics; Computational Mathematics and Num...,Springer New York,Springer
3,Social Anxiety and Social Phobia in Youth,Christopher Kearney,2005,Graduate/advanced undergraduate textbook,2005,Springer-Verlag US,978-0-387-22591-3,978-0-387-22592-0,EN,English/International,...,NaN,NaN,NaN,Series in Anxiety and Related Disorders,NaN,http://doi.org/10.1007/b99417,http://link.springer.com/openurl?genre=book&is...,Psychology; Clinical Psychology; Personality a...,Springer US,Springer
4,Discrete Mathematics,"László Lovász, József Pelikán, Katalin Veszter...",2003,Undergraduate textbook,2003,Springer-Verlag New York,978-0-387-95584-1,978-0-387-21777-2,EN,English/International,...,NaN,0172-6056,2197-5604,Undergraduate Texts in Mathematics,NaN,http://doi.org/10.1007/b97469,http://link.springer.com/openurl?genre=book&is...,Mathematics; Combinatorics; Number Theory,Springer New York,Springer


In [3]:
open_urls = np.array(df['OpenURL'])
book_titles = np.array(df['Book Title'])
years = np.array(df['Copyright Year']).astype('str') 
# alt = book_titles + '_' + years

In [4]:
start_url = 'https://link.springer.com'
download_urls = []

for i in range(len(open_urls)):
    print(i)
    site = open_urls[i]
    bookpage = requests.get(site)
    soup = BeautifulSoup(bookpage.text, "html.parser")
    try:
        end_url = soup.find('a', class_ = "test-bookpdf-link")['href']
        download_url = start_url+end_url
        download_urls.append(download_url)        
    except:
        print(site)
        download_urls.append(None)

df_2 = pd.DataFrame(download_urls)

0
1
2
3
4
5
6
7
8
9
10
http://link.springer.com/openurl?genre=book&isbn=978-0-387-36601-2
11
12
13
http://link.springer.com/openurl?genre=book&isbn=978-0-387-45524-2
14
15
http://link.springer.com/openurl?genre=book&isbn=978-0-387-46312-4
16
17
http://link.springer.com/openurl?genre=book&isbn=978-0-387-68566-3
18
19
20
21
22
23
24
25
26
27
28
29
30
31
http://link.springer.com/openurl?genre=book&isbn=978-1-4020-5719-9
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
http://link.springer.com/openurl?genre=book&isbn=978-1-4615-1077-2
54
http://link.springer.com/openurl?genre=book&isbn=978-3-319-02099-0
55
56
57
58
59
60
61
62
63
64
65
66
http://link.springer.com/openurl?genre=book&isbn=978-3-642-23026-4
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
http://link.springer.com/openurl?genre=book&isbn=978-3-319-54413-7
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126

In [5]:
# dirName = 'assets'
# Create target Directory if don't exist
# if not os.path.exists(dirName):
#     os.mkdir(dirName)
#     print("Directory " , dirName ,  " Created ")
# else:    
#     print("Directory " , dirName ,  " already exists")
logo_urls = []

for i in range(len(open_urls)):
    site = open_urls[i]
    bookpage = requests.get(site)
    soup = BeautifulSoup(bookpage.text, "html.parser")
    logo_url = soup.find('img', class_ = "test-cover-image")['src']
    logo_urls.append(logo_url)
    print(i)

logo_urls_df = pd.DataFrame(logo_urls)
#     r = requests.get(logo_url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

#     if r.status_code == 200:
#         try:
#             with open(dirName+'/'+alt[i]+'.jpg', 'wb') as f:
#                 r.raw.decode_content = True
#                 shutil.copyfileobj(r.raw, f)            
#         except:
#             print(site)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
logo_urls_df.head()
logo_urls_df.columns = ['logo URL']
df_2.columns = ['Download URL']
# df_3 = pd.read_json("Free+English+textbooks+with+download_url-1.json", orient='index')
df_3 = df.join(df_2, how="left")
df_4 = df_3.join(logo_urls_df, how="left")
df_5 = df_4[~df_4["Download URL"].isnull()].reset_index(drop=True)
df_5.head()

,Book Title,Author,Edition,Product Type,Copyright Year,Copyright Holder,Print ISBN,Electronic ISBN,Language,Language Collection,...,Series Electronic ISSN,Series Title,Volume Number,DOI URL,OpenURL,Subject Classification,Publisher,Imprint,Download URL,logo URL
0,Fundamentals of Power Electronics,"Robert W. Erickson, Dragan Maksimovic",2nd ed. 2001,Graduate/advanced undergraduate textbook,2001,Springer Science+Business Media New York,978-0-7923-7270-7,978-0-306-48048-5,EN,English/International,...,NaN,NaN,NaN,http://doi.org/10.1007/b100747,http://link.springer.com/openurl?genre=book&is...,"Engineering; Circuits and Systems; Energy, gen...",Springer US,Springer,https://link.springer.com/content/pdf/10.1007%...,https://media.springernature.com/w306/springer...
1,Handbook of the Life Course,"Jeylan T. Mortimer, Michael J. Shanahan",2003,Graduate/advanced undergraduate textbook,2003,Springer Science+Business Media New York,978-0-306-47498-9,978-0-306-48247-2,EN,English/International,...,2542-839X,Handbooks of Sociology and Social Research,NaN,http://doi.org/10.1007/b100507,http://link.springer.com/openurl?genre=book&is...,"Social Sciences; Sociology, general; Clinical ...",Springer US,Springer,https://link.springer.com/content/pdf/10.1007%...,https://media.springernature.com/w306/springer...
2,All of Statistics,Larry Wasserman,2004,Graduate/advanced undergraduate textbook,2004,Springer-Verlag New York,978-0-387-40272-7,978-0-387-21736-9,EN,English/International,...,2197-4136,Springer Texts in Statistics,NaN,http://doi.org/10.1007/978-0-387-21736-9,http://link.springer.com/openurl?genre=book&is...,Mathematics; Computational Mathematics and Num...,Springer New York,Springer,https://link.springer.com/content/pdf/10.1007%...,https://media.springernature.com/w306/springer...
3,Social Anxiety and Social Phobia in Youth,Christopher Kearney,2005,Graduate/advanced undergraduate textbook,2005,Springer-Verlag US,978-0-387-22591-3,978-0-387-22592-0,EN,English/International,...,NaN,Series in Anxiety and Related Disorders,NaN,http://doi.org/10.1007/b99417,http://link.springer.com/openurl?genre=book&is...,Psychology; Clinical Psychology; Personality a...,Springer US,Springer,https://link.springer.com/content/pdf/10.1007%...,https://media.springernature.com/w306/springer...
4,Discrete Mathematics,"László Lovász, József Pelikán, Katalin Veszter...",2003,Undergraduate textbook,2003,Springer-Verlag New York,978-0-387-95584-1,978-0-387-21777-2,EN,English/International,...,2197-5604,Undergraduate Texts in Mathematics,NaN,http://doi.org/10.1007/b97469,http://link.springer.com/openurl?genre=book&is...,Mathematics; Combinatorics; Number Theory,Springer New York,Springer,https://link.springer.com/content/pdf/10.1007%...,https://media.springernature.com/w306/springer...


In [10]:
df_5.to_json(r'Free+English+textbooks+with+download_url.json', orient='index')

In [ ]:
df_5.to_excel("Free+English+textbooks+with+download_url.xlsx")